In [1]:
import torch
import os

from datasets.make_cner import make_cner

from datasets.datasets import CNERDataset

from transformers import BertTokenizer, BertConfig

# Get model tokenizer
PRETRAINED_MODEL_NAME = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
                 It can be downloaded at https://aka.ms/vs/16/release/vc_redist.x64.exe


In [ ]:

words = list()
entities = list()
res = list()

sen_index = 0


with open('data/cner/dev.char.bmes', 'r', encoding='utf-8') as f:

    for line in f:

        l = line.split(' ')

        if len(l)>1:
            w = l[0]
            e = l[1].replace('\n', '')

            words.append(w)
            entities.append(e)

            res.append((sen_index, w, e))
        else:
            sen_index += 1
            pass

    f.close()
    pass

res[:10]
# words[:180], entities[:180]

In [ ]:
entities = [val[2] for val in res]
entities_label = list(set(entities))


index2entities = {i:val for i, val in enumerate(entities_label)}
index2entities[len(index2entities)] = '[PAD]'
index2entities[len(index2entities)] = '[CLS]'
index2entities[len(index2entities)] = '[SEP]'
entities2index = {v:i for i, v in index2entities.items()}

index2entities, entities2index

In [ ]:
_res = [(i, w, entities2index[e]) for i, w, e in res]
_res[:10]

In [ ]:
idx = 2
tokens_sen = list()
tokens_ent = list()
for i, w, e in _res:
    if(i==idx):
        tokens_sen.append(w)
        tokens_ent.append(e)
    pass

# 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
word_pieces = ["[CLS]"]
# tokens_sen = self.tokenizer.tokenize(sen)
word_pieces += tokens_sen + ["[SEP]"]

entity_pieces = [entities2index["[CLS]"]]
entity_pieces += tokens_ent + [entities2index["[SEP]"]]

word_pieces, entity_pieces, len(word_pieces), len(entity_pieces)

In [ ]:
cner_dataset = CNERDataset('dev', tokenizer, 'data/cner/test.char.bmes')

In [ ]:

cner_dataset[0]

In [ ]:
index2entities[25],index2entities[26], cner_dataset.index2entities[25], cner_dataset.index2entities[26]

In [ ]:
from preprocessing import create_mini_batch
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

train_loader = DataLoader(
    cner_dataset, 
    batch_size=4, 
    collate_fn=create_mini_batch
    )

In [2]:
index2entities, entities2index = make_cner('data/cner/dev.char.bmes', _mode='r')

In [3]:
index2entities, entities2index

({1: 'B-LOC',
  2: 'M-CONT',
  3: 'B-ORG',
  4: 'E-ORG',
  5: 'E-LOC',
  6: 'B-EDU',
  7: 'B-PRO',
  8: 'E-TITLE',
  9: 'E-RACE',
  10: 'M-TITLE',
  11: 'O',
  12: 'M-ORG',
  13: 'S-NAME',
  14: 'B-RACE',
  15: 'E-CONT',
  16: 'B-TITLE',
  17: 'S-RACE',
  18: 'M-PRO',
  19: 'M-EDU',
  20: 'M-LOC',
  21: 'B-NAME',
  22: 'M-NAME',
  23: 'E-NAME',
  24: 'E-PRO',
  25: 'B-CONT',
  26: 'E-EDU',
  0: '[PAD]',
  27: '[CLS]',
  28: '[SEP]'},
 {'B-LOC': 1,
  'M-CONT': 2,
  'B-ORG': 3,
  'E-ORG': 4,
  'E-LOC': 5,
  'B-EDU': 6,
  'B-PRO': 7,
  'E-TITLE': 8,
  'E-RACE': 9,
  'M-TITLE': 10,
  'O': 11,
  'M-ORG': 12,
  'S-NAME': 13,
  'B-RACE': 14,
  'E-CONT': 15,
  'B-TITLE': 16,
  'S-RACE': 17,
  'M-PRO': 18,
  'M-EDU': 19,
  'M-LOC': 20,
  'B-NAME': 21,
  'M-NAME': 22,
  'E-NAME': 23,
  'E-PRO': 24,
  'B-CONT': 25,
  'E-EDU': 26,
  '[PAD]': 0,
  '[CLS]': 27,
  '[SEP]': 28})

In [4]:
from transformers import BertTokenizer, BertForTokenClassification
import torch

# Get model tokenizer
PRETRAINED_MODEL_NAME = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained(PRETRAINED_MODEL_NAME, return_dict=True)
inputs = tokenizer("但既然大神都這樣建議了，那肯定不會有錯的。", return_tensors="pt")
labels = torch.tensor([1] * inputs["input_ids"].size(1)).unsqueeze(0)  # Batch size 1


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-c

In [5]:
inputs, labels

({'input_ids': tensor([[ 101,  852, 3188, 4197, 1920, 4868, 6963, 6857, 3564, 2456, 6359,  749,
          8024, 6929, 5507, 2137,  679, 3298, 3300, 7097, 4638,  511,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]))

In [6]:
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

In [9]:
loss, logits, inputs

(tensor(0.4396, grad_fn=<NllLossBackward>),
 tensor([[[-0.2191, -0.0391],
          [ 0.1303, -0.6352],
          [-0.2937,  0.2079],
          [-0.2876, -0.1088],
          [-0.5426, -0.0326],
          [-0.8943,  0.2427],
          [-0.3415,  0.0529],
          [-0.8706, -0.1366],
          [-0.3502,  0.1141],
          [-0.5358, -0.0063],
          [-0.5272,  0.7046],
          [-0.1649,  0.4001],
          [-0.5383,  0.3421],
          [-0.3403,  0.0428],
          [-0.7319,  0.6270],
          [-0.7603,  0.5703],
          [-0.7431,  0.4733],
          [-0.8255,  0.9471],
          [-0.0938,  0.3644],
          [-0.8089, -0.5504],
          [-0.7775, -0.0595],
          [-0.7209,  0.3260],
          [-0.1873,  0.3622]]], grad_fn=<AddBackward0>),
 {'input_ids': tensor([[ 101,  852, 3188, 4197, 1920, 4868, 6963, 6857, 3564, 2456, 6359,  749,
          8024, 6929, 5507, 2137,  679, 3298, 3300, 7097, 4638,  511,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

inputs